In [6]:
import os
from fastai.vision.all import *
import pandas as pd
import torch
import torch.nn as nn

In [7]:
# Base path to the data folder
base_data_path = Path('data')

# Initialize an empty DataFrame to store all the data
combined_df = pd.DataFrame()

# List all the folders in the base directory
folders = [f for f in os.listdir(base_data_path) if os.path.isdir(base_data_path/f)]

# Loop through each folder and load CSVs and image paths
for folder in folders:
    folder_path = base_data_path/folder
    
    # Load the CSV file (assumed to be named 'controls.csv')
    csv_file = folder_path/'controls.csv'
    if csv_file.exists():
        df = pd.read_csv(csv_file)
        
        # Get all image files that exist in the folder
        image_files = {f for f in os.listdir(folder_path) if f.endswith('.jpg')}
        
        # Dynamically match each row with an actual image filename
        def get_image_filename(i):
            # Construct the expected filename
            filename = f'frame_{i}.jpg'
            # Only return the filename if it exists in the folder
            if filename in image_files:
                return folder_path/filename
            else:
                return None

        # Apply the function to create a new column with actual image paths
        df['frame'] = df.index.map(get_image_filename)
        
        # Filter out rows where no corresponding image file was found
        df = df[df['frame'].notnull()]
        
        # Combine with the main DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# If combined_df is empty, raise an error
if combined_df.empty:
    raise FileNotFoundError("No valid image files found. Please check your data folders.")

# Define a function to get the control inputs for each image (as a tensor)
def get_y(row):
    return tensor([row['steering'], row['throttle'], row['brake']])

# Define a function to get the image path for each row
def get_x(row):
    return row['frame']  # Path is already constructed in 'combined_df'

# DataBlock for regression task
dblock = DataBlock(
    blocks=(ImageBlock, RegressionBlock),  # Images as input, continuous labels as output
    get_x=get_x,  # Get image path
    get_y=get_y,  # Get control inputs as a tensor
    splitter=RandomSplitter(valid_pct=0.3),  # Split train/validation
)

# Create DataLoaders
dls = dblock.dataloaders(combined_df)

In [18]:

nfeatures = 512 # resnet34 has 512 features before the head
features_out = 3 # throttle,steering,brake
body = create_body(resnet34(), pretrained=True)

head = create_head(nfeatures, features_out, ps=0.3) # ps is dropout rate

model = nn.Sequential(body,head)




In [12]:
# Define the learner with a pretrained ResNet model
learn = vision_learner(
    dls, 
    model, 
    loss_func=MSELossFlat(),
    y_range=(-1., 1.)  # Adjust as needed based on the range of your control inputs
)

learn.freeze()

learn.to_fp16()

# Fine-tune the model
print("Training the model")
learn.fine_tune(5)

# Evaluate the model on the validation set
print("Evaluating the model")
learn.validate()

# Save the entire model for future use
learn.export('trackmania_learner_fp16_custom_head.pkl')


TypeError: Sequential.forward() got an unexpected keyword argument 'pretrained'